# Описание проекта:
#### На основе данных о поведении клиентов необходимо построить модель классификации, способную подобрать для клиентов телеком компании наиболее подходящий тариф. Метрика accuracy должна быть не меньше 0.75.

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.



## Импорт библиотек:

In [ ]:
# Импортируем pandas как pd:
import pandas as pd
# Импортируем numpy как np:
import numpy as np
# Импортируем math:
import math
# Импорт библиотеки matplotlib.pyplot как plt:
import matplotlib.pyplot as plt
# Импорт разных моделей обучения, метод разделения данных, и метод проверки обучения:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Изучение общей информации о данных:

In [ ]:
# загружаем таблицу данных в переменную data:
try:
    data = pd.read_csv('datasets/users_behavior.csv')
except:
    data = pd.read_csv("/content/drive/MyDrive/For_data/users_behavior.csv")
# Выводим информацию о таблице данных:
data.info()
# Выводим таблицу:
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


### Вывод:
Дана таблица с 5 столбцами. Пропуски отсутсвуют во всей таблице, все данные численные.

сalls — количество звонков

minutes — суммарная длительность звонков в минутах

messages — количество sms-сообщений

mb_used — израсходованный интернет-трафик в Мб

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<div class="alert alert-block alert-success">
<b>Успех:</b> Ты хорошо владеешь методами первичного изучения данных 👍 Спасибо за информативный вывод
</div>

## Разделение исходных данных:

In [ ]:
# Создание таблицы с признаками:
features = data.drop(['is_ultra'], axis=1)
# Создание целевого столбца:
target = data['is_ultra']

# Разделение исходных данных на выборку для обучения 60%, и для валидации и теста:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, target, test_size=0.4, random_state=12345)
# Разделение выборки для валидации и теста, на выборку валидации 20% от общих данных(50% от того, что осталось)
# и на выборку теста 20% от общих данных ( 50% от того, что осталось после первого разделения):
features_valid, features_test, target_valid, target_test =  train_test_split(features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

#### Вывод:
Все данные я разделил на 3 выборки: обучающая - 60 %, выборка для валидации - 20% и выборка для теста - 20%.(3/1/1).

Для этого я сначала разделил данные на 2 выборки: обучающую - 60%,  и оставшуюся 40%, которую впоследствии я разделил на 2, чтобы получить по 20% для валидации и для тестов.

# Исcледование разных моделий:

## Дерево решений:

In [ ]:
# Цикл для проверки нескольких вариантов деревьев(с разной макс глубиной( от 1 до 10))
for depth in range(1, 11):
    # Обьявление модели:
    model = model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    # Обучение модели
    model.fit(features_train, target_train)
    # Предсказание:
    predictions_valid = model.predict(features_valid)
    # Вывод:
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


## Random forest:

In [ ]:
# Переменные чтобы индитефициовать лучшую модель:
best_model = None
best_result = 0
# Цикл для проверки разных "лесов"
for est in range(1, 50):
    # Обьявление модели с заданным кол-вом деревьев:
    model = RandomForestClassifier(random_state=12345, n_estimators= est)
    # Обучение модели:
    model.fit(features_train, target_train)
    # Качество модели на валидационной выборке:
    result = model.score(features_valid, target_valid)
    # Для поиска лучшей модели:
    if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
# Вывод:
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314 Количество деревьев: 23 Максимальная глубина: 10


## Логистическая регрессия:

In [ ]:
# Обьявление модели:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
# Обучение модели:
model.fit(features, target)
# Вывод качества модели
print(model.score(features_valid, target_valid))

0.7573872472783826


#### Вывод:
Данное исследование показало:

Самой успешной моделью для этих данных оказалось Random Forest(кол-во деревьев 23, максимальная глубина 5) с accuaracy равной '0.7947122861586314'.

Далее идёт Дерево решений с максимальной глубиной 3, accuaracy равно '0.7853810264385692'.

И Логическая регрессия с accuaracy равной '0.7573872472783826', но зато она грузилась быстрее всех(хотя тут по скорости отличались милисекунды всё равно было заметно)

Все модели проходят порог в accuaracy = 0.75 .

# Проверка качества моделей:

In [ ]:
# Дерево решений:
model = DecisionTreeClassifier(random_state=12345, max_depth = 3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('Дерево решений:',result)

# Random Forest:
model = RandomForestClassifier(random_state=12345, n_estimators=23, max_depth=5)
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
result = model.score(features_valid, target_valid)
print('Random forest:',result)

# Логическая регрессия:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print('Логическая регрессия:', result)

Дерево решений: 0.7791601866251944
Random forest: 0.7884914463452566
Логическая регрессия: 0.6842923794712286


### Вывод:
Как и в исследовании так и после проверки позиции остаются теми же:

Самой успешной моделью казалось Random Forest(кол-во деревьев 23, максимальная глубина 5) с accuaracy равной '0.7884914463452566', что уменьшилось незначительно по сравнению с исследованием, всего на чуть меньше чем 0.01.

Далее идёт Дерево решений с максимальной глубиной 3, accuaracy равно '0.7791601866251944', что также не сильно потеряло в точности.

Последней идёт модель Логической регрессии, для которого accuaracy равно '0.6842923794712286', что значительно по сравнению с другими моделями потеряло точость, видимо модель переобучена.

# Вывод:

##  Лучшей моделью оказалась Random forest, с кол-вом деревьев 23 и макс глубиной 5.

## В данном проекте были предприняты следующие шаги:

### 1.) Полученна и анализирована общая информация о данных.

### 2.) Данные разделены на 3 выборки: обучающую, валидационную и тестовую.

### 3.) Проведенно исследование 3х моделий обучений, выяснено какая из них лучше.

### 4.) Качество моделей из исследования проверено с помощью тестовой выборки, исследование потвердилось.
